# ml4t.backtest Capabilities Demo

This notebook showcases the full feature set of **ml4t.backtest**:

1. **Core Backtesting**: Event-driven execution
2. **Risk Management**: Stop-loss, take-profit, trailing stops
3. **Portfolio Rebalancing**: Weight-based allocation
4. **Execution Models**: Commission, slippage, market impact
5. **Analysis Integration**: Trade statistics, performance metrics

In [ ]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl

# Load ETF universe
DATA_PATH = Path.home() / "Dropbox" / "ml4t" / "data" / "etfs" / "etf_universe.parquet"
df_full = pl.read_parquet(DATA_PATH)

print(f"ETF Universe: {df_full['symbol'].n_unique()} symbols, {len(df_full):,} bars")

## 1. Basic Strategy with Risk Rules

In [ ]:
from ml4t.backtest import (
    BacktestAnalyzer,
    DataFeed,
    Engine,
    ExecutionMode,
    OrderSide,
    PercentageCommission,
    PercentageSlippage,
    Strategy,
)
from ml4t.backtest.risk.position.static import StopLoss, TakeProfit, TrailingStop


class MomentumWithRisk(Strategy):
    """Momentum strategy with comprehensive risk management."""

    def __init__(self, asset, lookback=20):
        self.asset = asset
        self.lookback = lookback
        self.prices = []

    def on_data(self, timestamp, data, context, broker):
        if self.asset not in data:
            return

        close = data[self.asset]["close"]
        self.prices.append(close)

        if len(self.prices) > self.lookback:
            self.prices.pop(0)

        if len(self.prices) < self.lookback:
            return

        # Calculate momentum
        momentum = (close / self.prices[0] - 1) * 100

        pos = broker.get_position(self.asset)
        has_position = pos is not None and pos.quantity > 0

        # Entry: Strong positive momentum
        if momentum > 5.0 and not has_position:
            equity = broker.get_account_value()
            qty = (equity * 0.95) / close
            broker.submit_order(self.asset, qty, OrderSide.BUY)

        # Exit: Negative momentum (risk rules also active)
        elif momentum < -3.0 and has_position:
            broker.submit_order(self.asset, pos.quantity, OrderSide.SELL)

In [ ]:
# Prepare data
ASSET = "SPY"
df = (
    df_full.filter(pl.col("symbol") == ASSET)
    .filter(pl.col("timestamp") >= datetime(2018, 1, 1))
    .filter(pl.col("timestamp") <= datetime(2023, 12, 31))
    .sort("timestamp")
    .rename({"symbol": "asset"})
)

print(f"{ASSET}: {len(df)} bars")

In [ ]:
# Create engine with risk rules
feed = DataFeed(prices_df=df)
strategy = MomentumWithRisk(ASSET, lookback=20)

# Define risk rules
position_rules = [
    StopLoss(stop_pct=0.05),  # 5% stop-loss
    TakeProfit(target_pct=0.15),  # 15% take-profit
    TrailingStop(trail_pct=0.08),  # 8% trailing stop
]

engine = Engine(
    feed=feed,
    strategy=strategy,
    initial_cash=100_000.0,
    execution_mode=ExecutionMode.NEXT_BAR,
    commission_model=PercentageCommission(0.001),  # 0.1% commission
    slippage_model=PercentageSlippage(0.001),  # 0.1% slippage
    position_rules=position_rules,
)

result = engine.run()

print("Strategy with Risk Management:")
print(f"  Final Value:  ${result['final_value']:,.2f}")
print(f"  Total Return: {result['total_return']:.2%}")
print(f"  Sharpe Ratio: {result['sharpe']:.2f}")
print(f"  Max Drawdown: {result['max_drawdown_pct']:.2%}")

In [ ]:
# Analyze trades
analyzer = BacktestAnalyzer(engine)
stats = analyzer.trade_statistics()
print(stats.summary())

## 2. Multi-Asset Portfolio Rebalancing

In [ ]:
from ml4t.backtest import RebalanceConfig, TargetWeightExecutor


class EqualWeightRebalance(Strategy):
    """Equal-weight portfolio with monthly rebalancing."""

    def __init__(self, assets, rebalance_freq=21):
        self.assets = assets
        self.rebalance_freq = rebalance_freq
        self.bar_count = 0

        self.executor = TargetWeightExecutor(
            config=RebalanceConfig(
                min_trade_value=500,
                min_weight_change=0.02,
                allow_fractional=True,
            )
        )

    def on_data(self, timestamp, data, context, broker):
        self.bar_count += 1

        # Rebalance monthly
        if self.bar_count % self.rebalance_freq != 1:
            return

        # Equal weights for available assets
        available = [a for a in self.assets if a in data]
        if not available:
            return

        weight = 0.95 / len(available)  # 95% invested, 5% cash buffer
        target_weights = {a: weight for a in available}

        # Execute rebalance
        orders = self.executor.execute(target_weights, data, broker)

        if orders:
            print(f"[{timestamp.date()}] Rebalanced to {len(available)} assets")

In [ ]:
# Multi-asset data
ASSETS = ["SPY", "TLT", "GLD", "EEM", "IWM"]

df_multi = (
    df_full.filter(pl.col("symbol").is_in(ASSETS))
    .filter(pl.col("timestamp") >= datetime(2015, 1, 1))
    .filter(pl.col("timestamp") <= datetime(2023, 12, 31))
    .sort(["timestamp", "symbol"])
    .rename({"symbol": "asset"})
)

print(f"Multi-asset data: {len(df_multi):,} bars")
print(f"Assets: {df_multi['asset'].unique().to_list()}")

In [ ]:
# Run portfolio backtest
feed = DataFeed(prices_df=df_multi)
strategy = EqualWeightRebalance(ASSETS, rebalance_freq=21)

engine = Engine(
    feed=feed,
    strategy=strategy,
    initial_cash=100_000.0,
    execution_mode=ExecutionMode.NEXT_BAR,
)

result = engine.run()

print("\nPortfolio Results:")
print(f"  Final Value:  ${result['final_value']:,.2f}")
print(f"  Total Return: {result['total_return']:.2%}")
print(f"  Sharpe Ratio: {result['sharpe']:.2f}")

## 3. Execution Models

In [ ]:
from ml4t.backtest import (
    # Commission models
    NoCommission,
    NoSlippage,
    PercentageCommission,
    PercentageSlippage,
)

print("Available Commission Models:")
print("  - NoCommission(): Zero commission")
print("  - PercentageCommission(0.001): 0.1% of trade value")
print("  - PerShareCommission(0.01): $0.01 per share")
print("  - TieredCommission(tiers): Volume-based tiers")
print()
print("Available Slippage Models:")
print("  - NoSlippage(): Zero slippage")
print("  - FixedSlippage(0.01): $0.01 per share")
print("  - PercentageSlippage(0.001): 0.1% of price")
print("  - VolumeShareSlippage(factor): Volume-dependent")
print()
print("Available Market Impact Models:")
print("  - LinearImpact(coef): Price impact = coef × participation")
print("  - SquareRootImpact(coef): Price impact = coef × √participation")

In [ ]:
# Compare execution cost scenarios

scenarios = [
    ("No costs", NoCommission(), NoSlippage()),
    ("Low cost (0.05%)", PercentageCommission(0.0005), PercentageSlippage(0.0005)),
    ("Medium cost (0.1%)", PercentageCommission(0.001), PercentageSlippage(0.001)),
    ("High cost (0.2%)", PercentageCommission(0.002), PercentageSlippage(0.002)),
]

# Single-asset data for comparison
df_spy = (
    df_full.filter(pl.col("symbol") == "SPY")
    .filter(pl.col("timestamp") >= datetime(2020, 1, 1))
    .filter(pl.col("timestamp") <= datetime(2023, 12, 31))
    .sort("timestamp")
    .rename({"symbol": "asset"})
)

results = []
for name, comm, slip in scenarios:
    feed = DataFeed(prices_df=df_spy)
    strategy = MomentumWithRisk("SPY", lookback=20)

    engine = Engine(
        feed=feed,
        strategy=strategy,
        initial_cash=100_000.0,
        execution_mode=ExecutionMode.NEXT_BAR,
        commission_model=comm,
        slippage_model=slip,
    )

    result = engine.run()
    results.append((name, result["final_value"], result["total_return"]))

# Display comparison
comparison = pd.DataFrame(results, columns=["Scenario", "Final Value", "Total Return"])
comparison["Final Value"] = comparison["Final Value"].apply(lambda x: f"${x:,.2f}")
comparison["Total Return"] = comparison["Total Return"].apply(lambda x: f"{x:.2%}")

print("\nExecution Cost Impact:")
print(comparison.to_string(index=False))

## 4. Trade Analysis

In [ ]:
# Run a strategy for detailed analysis
feed = DataFeed(prices_df=df_spy)
strategy = MomentumWithRisk("SPY", lookback=20)

engine = Engine(
    feed=feed,
    strategy=strategy,
    initial_cash=100_000.0,
    execution_mode=ExecutionMode.NEXT_BAR,
)

result = engine.run()

# Create analyzer
analyzer = BacktestAnalyzer(engine)

In [ ]:
# Trade statistics
stats = analyzer.trade_statistics()
print("=" * 50)
print("TRADE STATISTICS")
print("=" * 50)
print(stats.summary())

In [ ]:
# Get trades as DataFrame
trades_df = analyzer.get_trades_dataframe()

if len(trades_df) > 0:
    print(f"\nTotal trades: {len(trades_df)}")
    print("\nBest trade:")
    best = trades_df.sort("pnl", descending=True).head(1).row(0, named=True)
    print(f"  {best['entry_time'].date()} → {best['exit_time'].date()}")
    print(f"  PnL: ${best['pnl']:,.2f} ({best['pnl_percent']:.2%})")
    print(f"  Bars held: {best['bars_held']}")

    print("\nWorst trade:")
    worst = trades_df.sort("pnl").head(1).row(0, named=True)
    print(f"  {worst['entry_time'].date()} → {worst['exit_time'].date()}")
    print(f"  PnL: ${worst['pnl']:,.2f} ({worst['pnl_percent']:.2%})")

In [ ]:
# Equity curve
import matplotlib.pyplot as plt

equity = analyzer.equity_history
timestamps = (
    engine.broker._timestamps if hasattr(engine.broker, "_timestamps") else list(range(len(equity)))
)

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Equity curve
axes[0].plot(timestamps, equity, linewidth=1.5)
axes[0].axhline(y=100000, color="gray", linestyle="--", alpha=0.5, label="Initial Capital")
axes[0].set_title("Equity Curve")
axes[0].set_ylabel("Portfolio Value ($)")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
equity_arr = np.array(equity)
running_max = np.maximum.accumulate(equity_arr)
drawdown = (equity_arr - running_max) / running_max * 100

axes[1].fill_between(range(len(drawdown)), drawdown, 0, alpha=0.3, color="red")
axes[1].plot(drawdown, color="red", linewidth=1)
axes[1].set_title("Drawdown")
axes[1].set_ylabel("Drawdown (%)")
axes[1].set_xlabel("Bar")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

**ml4t.backtest** provides:

| Feature | Description |
|---------|-------------|
| **Event-Driven** | Sequential bar processing, no look-ahead bias |
| **Execution Modes** | SAME_BAR and NEXT_BAR |
| **Risk Management** | Stop-loss, take-profit, trailing stops |
| **Portfolio Rules** | Max positions, max exposure, drawdown limits |
| **Rebalancing** | Weight-based allocation with TargetWeightExecutor |
| **Cost Models** | Commission, slippage, market impact |
| **Analysis** | Trade statistics, equity curves, performance metrics |
| **Data** | Polars-native, supports multi-asset |

### Validated Against
- ✅ VectorBT Pro (exact match)
- ✅ VectorBT OSS (exact match)
- ✅ Backtrader (exact match)
- ✅ Zipline-reloaded (within tolerance)